In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs
plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
airline_data = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/airline_data.csv", parse_dates = ["date"])
cleaned_reviews = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/cleaned-reviews.csv", parse_dates = ["date"]) # todos los que no opinaron se filtraron ????/
costumer_data = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/customer_airways_data.csv", encoding = "iso-8859-1")
costumer_data_filtrada = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/filtered_customer_booking.csv")

In [4]:
preferencias = costumer_data.iloc[:,8:12]
preferencias = preferencias.rename(columns={"booking_origin": "country"})

In [5]:
def corregir_rates(serie):
  if(serie == "None"):
    return -1
  elif(serie == "\n\t\t\t\t\t\t\t\t\t\t\t\t\t5"):
    return 5
  else:
    return serie

airline_data.rates = airline_data.rates.map(corregir_rates)
airline_data.rates = airline_data.rates.astype(np.int8)

In [ ]:
datos_filtrados = airline_data.iloc[:,[2,4]]
datos_encuesta = pd.merge(datos_filtrados,preferencias,how = "outer")

datos_encuesta_validos =datos_encuesta[datos_encuesta.rates > 0 ]#consideramos que 9 es un puntaje alto
datos_encuesta_completos = datos_encuesta_validos.dropna()
datos_encuesta_completos["country"].value_counts()

conteo_paises = datos_encuesta_completos['country'].value_counts().reset_index()
conteo_paises.columns = ['country', 'conteo_pais']
conteo_paises = conteo_paises.head(5)
conteo_paises

In [ ]:
promedio_por_paises = datos_encuesta_completos.groupby('country')[['wants_in_flight_meals', 'wants_extra_baggage', 'wants_preferred_seat', 'rates']].mean().reset_index()

promedio_por_paises = promedio_por_paises.sort_values("rates", ascending=False)

paises_mas_encuestas_con_promedios = conteo_paises.merge(promedio_por_paises, on='country', how='inner')
paises_mas_encuestas_con_promedios = paises_mas_encuestas_con_promedios.iloc[:,[0,2,3,4,5]]
paises_mas_encuestas_con_promedios

In [ ]:
paises_mas_encuestas_con_promedios.set_index('country', inplace=True)
sns.heatmap(paises_mas_encuestas_con_promedios, annot=True, cmap="YlGnBu", fmt="g")

plt.title('Heatmap de preferencias por país')
plt.xlabel('Promedio de preferencia por pais')
plt.ylabel('Países')

# Mostramos el heatmap
plt.show()

In [220]:
#esstadistica de rutas preferidas segun fecha -> clasificar fecha por cuatrimestre #suponemos que reservan desde su pais de origen
informacion_de_compra = costumer_data_filtrada[["route", "trip_type", "sales_channel", "booking_complete", "purchase_lead", "booking_origin"]]
informacion_de_compra = informacion_de_compra.rename(columns={"booking_origin": "country"})
fecha_de_viaje = cleaned_reviews[["date", "verified", "country"]]
informacion_de_vuelo = pd.merge(informacion_de_compra, fecha_de_viaje, how="outer")
informacion_de_vuelo = informacion_de_vuelo.dropna()
vuelos_validos = informacion_de_vuelo.loc[(informacion_de_vuelo.booking_complete == 1) & (informacion_de_vuelo.verified == True)]

In [221]:
def mes_a_cuatrimestre(fecha):
  if 1 <= fecha.month <= 3:
    return 1
  elif 4 <= fecha.month <= 6:
    return 2
  elif 7 <= fecha.month <= 9:
    return 3
  else:
    return 4

vuelos_validos.date = vuelos_validos.date.map(mes_a_cuatrimestre)
vuelos_validos = vuelos_validos.rename(columns={"date": "quarter"})



In [265]:
#agrupamos por pais de origen -> vemos que ruta es la mas usada para cada pais y que tipo de venta se uso

paises_y_rutas = vuelos_validos.groupby(["country", "route","trip_type"]).size().reset_index()
paises_y_rutas = pd.DataFrame(paises_y_rutas)
paises_y_rutas["cantidad_personas"] = paises_y_rutas.iloc[:,3]
paises_y_rutas = paises_y_rutas.iloc[:,[0,1,2,4]]
paises_y_rutas


#ponemos los 5 paises mas visitados y que tipo de viaje se uso y que ruta




,country,route,trip_type,cantidad_personas
0,Australia,AKLKUL,RoundTrip,120
1,Australia,AORMEL,RoundTrip,60
2,Australia,BBISYD,RoundTrip,60
3,Australia,BDOMEL,RoundTrip,60
4,Australia,BDOOOL,RoundTrip,30
...,...,...,...,...
1020,Vietnam,MRUSIN,RoundTrip,1
1021,Vietnam,OOLSGN,RoundTrip,10
1022,Vietnam,PERSGN,RoundTrip,12
1023,Vietnam,PVGSGN,RoundTrip,2
